# Astra DB Serverless, Python Quickstart

This notebook accompanies the [Astra DB Quickstart](https://docs.datastax.com/en/astra-db-serverless/get-started/quickstart.html).

The differences are:

- This notebook reads secrets from the notebook instead of from environment variables.
- This notebook hardcodes the data to insert instead of reading the data from a file.

## Create a database and store your credentials


1. Follow the instructions in the [quickstart](https://docs.datastax.com/en/astra-db-serverless/get-started/quickstart.html) to create a database and get the credentials.
1. Open the Secrets on the left sidebar in this notebook (the Key icon).
  1. Store your endpoint in a secret called `ASTRA_DB_API_ENDPOINT`.
  1. Store your token in a secret called `ASTRA_DB_APPLICATION_TOKEN`.
  1. Ensure that the "Notebook Access" toggle is enabled for both secrets.

## Install the client

In [ ]:
!pip install "astrapy>=2.0,<3.0"

## Import libraries

In [ ]:
import json
from typing import Any, Callable, Dict, List

from google.colab import userdata

from astrapy import Collection, DataAPIClient, Database
from astrapy.constants import VectorMetric
from astrapy.info import (
    CollectionDefinition,
    CollectionVectorOptions,
    VectorServiceOptions,
)

## Get the data to insert

In [ ]:
json_data = [{'title': 'Hidden Shadows of the Past', 'author': 'John Anthony', 'numberOfPages': 481, 'rating': 1.0, 'publicationYear': 2002, 'summary': "Set against a forgotten metropolis, 'Hidden Shadows of the Past' by John Anthony unveils a journey to the underworld. The story builds through defying a cruel god, offering a gripping tale of suspense.", 'genres': ['Biography', 'Graphic Novel', 'Dystopian', 'Drama'], 'metadata': {'ISBN': '978-1-905585-40-3', 'language': 'French', 'edition': 'Anniversary Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Beyond Moments of Broken Promises', 'author': 'Daniel Larson', 'numberOfPages': 704, 'rating': 3.7, 'publicationYear': 1978, 'summary': "Set against a secret base in the Arctic, 'Beyond Moments of Broken Promises' by Daniel Larson unveils a search for inner peace. The story builds through rediscovering a lost friendship, offering a thrilling and action-packed blockbuster.", 'genres': ['Dystopian'], 'metadata': {'ISBN': '978-0-88734-027-7', 'language': 'Japanese', 'edition': 'Anniversary Edition'}, 'isCheckedOut': True, 'borrower': 'Alexander Rodriguez', 'dueDate': '2024-12-18'}, {'title': 'Within Echoes Through Time', 'author': 'Ryan Jackson', 'numberOfPages': 714, 'rating': 2.4, 'publicationYear': 1993, 'summary': "'Within Echoes Through Time' by Ryan Jackson immerses readers in a magical forest, where a quest for the truth collides with navigating treacherous terrain, delivering a rich and immersive world.", 'genres': ['Tragedy', 'Fantasy', 'Satire'], 'metadata': {'ISBN': '978-0-02-313022-9', 'language': 'Spanish', 'edition': 'Illustrated Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'The Flames of Eternal Night', 'author': 'Deanna Fisher', 'numberOfPages': 198, 'rating': 4.6, 'publicationYear': 1996, 'summary': "Deanna Fisher's 'The Flames of Eternal Night' is a journey to the edge of the world that takes place in a forgotten metropolis. This is a a deeply moving story that keep you reading while the characters are redefining life.", 'genres': ['Dystopian', 'Adventure'], 'metadata': {'ISBN': '978-1-5358-6857-0', 'language': 'German', 'edition': 'Illustrated Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Against Legends of Brightness', 'author': 'Christopher Washington', 'numberOfPages': 327, 'rating': 1.1, 'publicationYear': 1975, 'summary': "'Against Legends of Brightness' is Christopher Washington's brilliant novel, set in a hidden library. Exploring a battle for the future, the characters face unraveling a dark secret, resulting in an unforgettable and immersive experience.", 'genres': ['Memoir', 'History'], 'metadata': {'ISBN': '978-0-11-359399-6', 'language': 'Spanish', 'edition': 'Third'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Beyond Waves that Changed Everything', 'author': 'Jeffrey Davis', 'numberOfPages': 185, 'rating': 4.3, 'publicationYear': 2020, 'summary': "In the backdrop of a futuristic laboratory, 'Beyond Waves that Changed Everything' by Jeffrey Davis explores a journey of self-discovery. The characters' journey through surviving against all odds results in a thrilling and fast-paced adventure.", 'genres': ['Biography', 'Epic'], 'metadata': {'ISBN': '978-0-309-98069-2', 'language': 'Italian', 'edition': 'Anniversary Edition'}, 'isCheckedOut': True, 'borrower': 'James Hanson', 'dueDate': '2024-11-28'}, {'title': 'Into Shadows of Tomorrow', 'author': 'Nicole Wright', 'numberOfPages': 598, 'rating': 3.1, 'publicationYear': 2020, 'summary': "Set against a distant alien world, 'Into Shadows of Tomorrow' by Nicole Wright unveils an unraveling conspiracy. The story builds through forging unexpected alliances, offering a dark and brooding story.", 'genres': ['Thriller', 'Philosophy', 'Fantasy'], 'metadata': {'ISBN': '978-1-4715-0356-6', 'language': 'Italian', 'edition': 'Anniversary Edition'}, 'isCheckedOut': True, 'borrower': 'Gregory Adams', 'dueDate': '2024-12-05'}, {'title': 'Beyond Tides of a Lifetime', 'author': 'Kendra Knight', 'numberOfPages': 356, 'rating': 1.9, 'publicationYear': 1987, 'summary': "'Beyond Tides of a Lifetime' is Kendra Knight's latest masterpiece set in a distant alien world. With a focus on a search for meaning, the story brings to life unraveling a dark secret, delivering a lighthearted romp.", 'genres': ['Thriller', 'History', 'Epic', 'Philosophy'], 'metadata': {'ISBN': '978-0-295-38479-5', 'language': 'Italian', 'edition': 'Second'}, 'isCheckedOut': True, 'borrower': 'Leslie Hansen', 'dueDate': '2024-12-05'}, {'title': 'The Shadows of Eternal Night', 'author': 'Martin Harris', 'numberOfPages': 198, 'rating': 1.0, 'publicationYear': 2007, 'summary': "Martin Harris's masterpiece, 'The Shadows of Eternal Night', is a story set in a crumbling mansion. Focusing on a struggle against darkness, the narrative reveals surviving against all odds, delivering a deeply introspective narrative.", 'genres': ['Thriller'], 'metadata': {'ISBN': '978-1-301-16043-3', 'language': 'Chinese', 'edition': 'Special Release'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Within the Horizon and the Last Frontier', 'author': 'Frank Costa', 'numberOfPages': 385, 'rating': 4.0, 'publicationYear': 2018, 'summary': "In a mysterious castle, 'Within the Horizon and the Last Frontier' by Frank Costa takes readers on a journey through a search for lost treasure. The characters are challenged by reconciling a bitter feud, creating a gripping and intense psychological drama.", 'genres': ['Dystopian', 'Adventure', 'Science Fiction', 'Non-Fiction'], 'metadata': {'ISBN': '978-1-138-81334-2', 'language': 'Japanese', 'edition': 'Anniversary Edition'}, 'isCheckedOut': True, 'borrower': 'Cody Harris', 'dueDate': '2024-12-18'}, {'title': 'The Shadows of Brightness', 'author': 'Sara Jones', 'numberOfPages': 185, 'rating': 2.9, 'publicationYear': 1995, 'summary': "Within the world of a crumbling mansion, Sara Jones's 'The Shadows of Brightness' unravels a tale of a journey to the afterlife. Characters must confront fulfilling a forgotten prophecy, resulting in a gripping and intense psychological drama.", 'genres': ['Philosophy', 'Memoir', 'Romance'], 'metadata': {'ISBN': '978-1-57611-473-5', 'language': 'English', 'edition': 'Second'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Under Secrets and the Unknown', 'author': 'Ashley Villanueva', 'numberOfPages': 225, 'rating': 3.4, 'publicationYear': 1992, 'summary': "Within a war-torn kingdom, 'Under Secrets and the Unknown' by Ashley Villanueva captures the spirit of a search for meaning. The story builds as characters confront escaping from relentless pursuers, offering a deeply moving story.", 'genres': ['Fantasy', 'Romance', 'Adventure'], 'metadata': {'ISBN': '978-0-261-26454-0', 'language': 'Korean', 'edition': 'Anniversary Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Under Waves Through Time', 'author': 'Nicholas Fischer', 'numberOfPages': 162, 'rating': 4.0, 'publicationYear': 1998, 'summary': "Nicholas Fischer's masterpiece, 'Under Waves Through Time', is a story set in a war-torn kingdom. Focusing on a battle for the future, the narrative reveals rebuilding a kingdom, delivering a gripping and intense psychological drama.", 'genres': ['Graphic Novel'], 'metadata': {'ISBN': '978-1-74188-720-4', 'language': 'Italian', 'edition': 'Special Release'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Beneath Notes of the Past', 'author': 'Mary Robinson', 'numberOfPages': 643, 'rating': 1.4, 'publicationYear': 1980, 'summary': "Set amidst a distant alien world, 'Beneath Notes of the Past' by Mary Robinson takes on a battle against the elements, where facing their greatest fears leads to a whimsical and enchanting fable.", 'genres': ['Satire'], 'metadata': {'ISBN': '978-1-65916-871-6', 'language': 'Chinese', 'edition': 'Second'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Beneath Whispers that Changed Everything', 'author': 'Sarah Lewis', 'numberOfPages': 690, 'rating': 4.1, 'publicationYear': 2024, 'summary': "In the backdrop of a futuristic laboratory, 'Beneath Whispers that Changed Everything' by Sarah Lewis explores a struggle for power. The characters' journey through confronting a powerful enemy results in a suspenseful page-turner.", 'genres': ['Horror'], 'metadata': {'ISBN': '978-1-76888-482-8', 'language': 'Italian', 'edition': 'Illustrated Edition'}, 'isCheckedOut': True, 'borrower': 'Samantha Davis', 'dueDate': '2024-12-02'}, {'title': 'In Secrets in the Dark', 'author': 'Anne Patrick', 'numberOfPages': 521, 'rating': 4.2, 'publicationYear': 2022, 'summary': "In 'In Secrets in the Dark', Anne Patrick weaves a tale set in an isolated lighthouse, centered around a fight for survival. With unleashing a terrible force, the story becomes a heartwarming journey.", 'genres': ['Non-Fiction'], 'metadata': {'ISBN': '978-0-218-35196-5', 'language': 'French', 'edition': 'Anniversary Edition'}, 'isCheckedOut': True, 'borrower': 'Charles Leach', 'dueDate': '2024-11-26'}, {'title': 'Among Storms on the Edge', 'author': 'Corey Hernandez', 'numberOfPages': 392, 'rating': 4.3, 'publicationYear': 1998, 'summary': "Set in an abandoned carnival, 'Among Storms on the Edge' by Corey Hernandez dives into a quest for peace. Facing defeating a powerful adversary, this is a evocative tapestry of emotions.", 'genres': ['Adventure', 'Philosophy'], 'metadata': {'ISBN': '978-1-65246-664-2', 'language': 'English', 'edition': 'Limited Edition'}, 'isCheckedOut': True, 'borrower': 'Ryan Reyes', 'dueDate': '2024-12-09'}, {'title': 'Under Ghosts in the Universe', 'author': 'Tanya Perez', 'numberOfPages': 345, 'rating': 4.6, 'publicationYear': 2001, 'summary': "'Under Ghosts in the Universe', a novel by Tanya Perez, is set in an underwater cave, where a battle for the ages unfolds. The characters grapple with rebuilding a shattered world, creating a dark and twisted tale of horror.", 'genres': ['Dystopian', 'Drama', 'Poetry', 'Comedy'], 'metadata': {'ISBN': '978-0-489-60732-7', 'language': 'Arabic', 'edition': "Collector's Edition"}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'From Operas and the Last Frontier', 'author': 'Yolanda Anderson', 'numberOfPages': 412, 'rating': 1.7, 'publicationYear': 2017, 'summary': "Yolanda Anderson's 'From Operas and the Last Frontier' is a quest for vengeance that takes place in a haunted shack. This is a a gripping and intense psychological drama that keep you reading while the characters are running for their lives.", 'genres': ['Drama', 'Adventure', 'Self-Help', 'Satire'], 'metadata': {'ISBN': '978-0-8461-2621-8', 'language': 'Spanish', 'edition': 'Deluxe Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Within the Horizon of the Heart', 'author': 'Randy Cook', 'numberOfPages': 171, 'rating': 4.2, 'publicationYear': 2015, 'summary': "In an abandoned carnival, 'Within the Horizon of the Heart' by Randy Cook takes readers on a journey through a battle for the future. The characters are challenged by defying a cruel god, creating a rich and evocative tapestry of words.", 'genres': ['Poetry', 'Biography'], 'metadata': {'ISBN': '978-0-7369-1813-8', 'language': 'Russian', 'edition': 'Deluxe Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'The Flames in the Dark', 'author': 'Jason Choi', 'numberOfPages': 595, 'rating': 3.3, 'publicationYear': 2009, 'summary': "'The Flames in the Dark' is Jason Choi's brilliant novel, set in a secret base in the Arctic. Exploring an exploration of forbidden knowledge, the characters face reconciling a betrayal, resulting in a lighthearted romp.", 'genres': ['History', 'Satire', 'Drama'], 'metadata': {'ISBN': '978-0-341-02285-5', 'language': 'Spanish', 'edition': 'First'}, 'isCheckedOut': True, 'borrower': 'Kelly Green', 'dueDate': '2024-12-04'}, {'title': 'Into Acquaintances Through Time', 'author': 'Cynthia Duncan', 'numberOfPages': 246, 'rating': 3.9, 'publicationYear': 2006, 'summary': "Set in a haunted shack, 'Into Acquaintances Through Time' by Cynthia Duncan dives into a search for purpose. Facing defeating a powerful adversary, this is a thrilling and fast-paced adventure.", 'genres': ['Science Fiction'], 'metadata': {'ISBN': '978-1-190-67288-8', 'language': 'German', 'edition': 'Deluxe Edition'}, 'isCheckedOut': True, 'borrower': 'Charles Rubio', 'dueDate': '2024-12-21'}, {'title': 'Hidden Moons on the Edge', 'author': 'Kristin Bell', 'numberOfPages': 725, 'rating': 1.8, 'publicationYear': 1975, 'summary': "In the backdrop of an abandoned carnival, 'Hidden Moons on the Edge' by Kristin Bell explores a quest for the truth. The characters' journey through unraveling a dark secret results in a thrilling and fast-paced adventure.", 'genres': ['Graphic Novel'], 'metadata': {'ISBN': '978-1-63808-366-5', 'language': 'Chinese', 'edition': 'Anniversary Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Beneath Voices of Yesterday', 'author': 'Rodney Hopkins', 'numberOfPages': 638, 'rating': 1.3, 'publicationYear': 1985, 'summary': "'Beneath Voices of Yesterday' by Rodney Hopkins is set in an oasis of art and tells a story of a journey to the stars. The protagonists face reclaiming a stolen legacy, in a tale that delivers a heartrending saga of love and loss.", 'genres': ['Romance', 'Thriller', 'Mystery', 'Young Adult'], 'metadata': {'ISBN': '978-1-65322-291-9', 'language': 'French', 'edition': 'Third'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'A Phantoms of the Heart', 'author': 'Steven Jackson', 'numberOfPages': 723, 'rating': 3.1, 'publicationYear': 1989, 'summary': "Set in a remote island, 'A Phantoms of the Heart' by Steven Jackson dives into an unraveling conspiracy. Facing rekindling a lost love, this is a deep character study.", 'genres': ['Non-Fiction'], 'metadata': {'ISBN': '978-1-883578-61-9', 'language': 'Korean', 'edition': 'Deluxe Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Beneath Waves of Yesterday', 'author': 'Tonya Kennedy', 'numberOfPages': 153, 'rating': 2.7, 'publicationYear': 1982, 'summary': "In a hidden library, Tonya Kennedy's 'Beneath Waves of Yesterday' unravels a gripping tale of a struggle for power. The characters must navigate defying a cruel fate, creating a gripping and intense psychological drama.", 'genres': ['Comedy'], 'metadata': {'ISBN': '978-0-19-555638-4', 'language': 'Japanese', 'edition': 'First'}, 'isCheckedOut': True, 'borrower': 'Carol Edwards', 'dueDate': '2024-12-03'}, {'title': 'Among Ghosts in the Dark', 'author': 'Michael Rojas', 'numberOfPages': 199, 'rating': 2.8, 'publicationYear': 2013, 'summary': "Set in a war-torn kingdom, 'Among Ghosts in the Dark' by Michael Rojas dives into an exploration of forbidden knowledge. Facing reviving a shattered empire, this is a thrilling and action-packed blockbuster.", 'genres': ['Graphic Novel', 'Mystery', 'Adventure', 'Dystopian'], 'metadata': {'ISBN': '978-1-255-54864-6', 'language': 'French', 'edition': 'First'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Within Light on the Edge', 'author': 'Kathy Vincent', 'numberOfPages': 166, 'rating': 3.1, 'publicationYear': 1985, 'summary': "Set against a hidden library, 'Within Light on the Edge' by Kathy Vincent unveils a search for lost treasure. The story builds through defending a fragile hope, offering a masterful blend of tension and hope.", 'genres': ['Self-Help'], 'metadata': {'ISBN': '978-0-9823322-8-3', 'language': 'Japanese', 'edition': 'Deluxe Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Under Storms of Lost Souls', 'author': 'Tammy Ramos', 'numberOfPages': 696, 'rating': 5.0, 'publicationYear': 1988, 'summary': "Within a sprawling museum, 'Under Storms of Lost Souls' by Tammy Ramos captures the spirit of a quest for the truth. The story builds as characters confront facing their greatest fears, offering a whimsical and enchanting fable.", 'genres': ['Poetry', 'Historical Fiction', 'Graphic Novel', 'Satire'], 'metadata': {'ISBN': '978-1-907161-02-5', 'language': 'Japanese', 'edition': "Collector's Edition"}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Against Light of Eternal Night', 'author': 'Jennifer Ray', 'numberOfPages': 655, 'rating': 4.8, 'publicationYear': 1999, 'summary': "'Against Light of Eternal Night', a novel by Jennifer Ray, is set in a mysterious castle, where a fight for love unfolds. The characters grapple with fulfilling a forgotten destiny, creating a whimsical and enchanting fable.", 'genres': ['Romance'], 'metadata': {'ISBN': '978-0-213-40965-4', 'language': 'Portuguese', 'edition': "Collector's Edition"}, 'isCheckedOut': True, 'borrower': 'Benjamin Franklin', 'dueDate': '2024-12-12'}, {'title': 'Under Operas of Tomorrow', 'author': 'Brittany Park', 'numberOfPages': 435, 'rating': 2.8, 'publicationYear': 1988, 'summary': "'Under Operas of Tomorrow', written by Brittany Park, is a story set in a mysterious castle. Delving into a battle for the future, the characters tackle unleashing a terrible weapon, creating a rich and evocative tapestry of words.", 'genres': ['Romance', 'Science Fiction', 'Fantasy', 'Self-Help'], 'metadata': {'ISBN': '978-0-524-06824-3', 'language': 'Korean', 'edition': 'Deluxe Edition'}, 'isCheckedOut': True, 'borrower': 'Elizabeth Atkinson', 'dueDate': '2024-12-15'}, {'title': 'A Operas of a Lifetime', 'author': 'Jonathan Solis', 'numberOfPages': 703, 'rating': 3.1, 'publicationYear': 2021, 'summary': "In Jonathan Solis's 'A Operas of a Lifetime', set in a mysterious castle, readers explore a quest for immortality. Faced with defending a dream, the story delivers a poetic and haunting story.", 'genres': ['Dystopian', 'History'], 'metadata': {'ISBN': '978-1-124-33296-3', 'language': 'Italian', 'edition': 'Second'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Under Echoes of the Heart', 'author': 'Kevin White', 'numberOfPages': 546, 'rating': 2.9, 'publicationYear': 1975, 'summary': "In an abandoned carnival, 'Under Echoes of the Heart' by Kevin White takes readers on a journey through an unraveling conspiracy. The characters are challenged by defending a fragile hope, creating a poignant and bittersweet reflection.", 'genres': ['Historical Fiction', 'Drama', 'Memoir', 'History'], 'metadata': {'ISBN': '978-1-75598-068-7', 'language': 'Italian', 'edition': 'Deluxe Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Through Darkness of Brightness', 'author': 'Darrell Vargas', 'numberOfPages': 740, 'rating': 4.6, 'publicationYear': 1997, 'summary': "Set amidst a crumbling mansion, 'Through Darkness of Brightness' by Darrell Vargas takes on a struggle against destiny, where finding a way home leads to a gripping tale of suspense.", 'genres': ['Epic', 'Biography', 'Thriller', 'Memoir'], 'metadata': {'ISBN': '978-0-06-985516-2', 'language': 'Chinese', 'edition': 'Anniversary Edition'}, 'isCheckedOut': True, 'borrower': 'Brian Lee', 'dueDate': '2024-12-10'}, {'title': 'Under Moments Through Time', 'author': 'Lawrence Kirby', 'numberOfPages': 337, 'rating': 1.6, 'publicationYear': 1999, 'summary': "In Lawrence Kirby's 'Under Moments Through Time', set in an underwater cave, readers explore a forbidden love affair. Faced with unleashing a terrible curse, the story delivers a rich and immersive world.", 'genres': ['Historical Fiction', 'Self-Help'], 'metadata': {'ISBN': '978-0-9879277-1-2', 'language': 'German', 'edition': 'First'}, 'isCheckedOut': True, 'borrower': 'Penny Hayes', 'dueDate': '2024-12-19'}, {'title': 'Under Friendships and the Journey', 'author': 'Edward Rojas', 'numberOfPages': 344, 'rating': 1.5, 'publicationYear': 1991, 'summary': "Set against an isolated lighthouse, 'Under Friendships and the Journey' by Edward Rojas unveils a battle for freedom. The story builds through unlocking a hidden power, offering a dark and twisted tale of horror.", 'genres': ['Romance'], 'metadata': {'ISBN': '978-1-84463-203-9', 'language': 'Korean', 'edition': "Collector's Edition"}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Within Silence and the Journey', 'author': 'Ashley Oliver', 'numberOfPages': 420, 'rating': 2.5, 'publicationYear': 2022, 'summary': "Within the world of a mysterious castle, Ashley Oliver's 'Within Silence and the Journey' unravels a tale of a struggle for power. Characters must confront facing their greatest fears, resulting in a evocative tapestry of emotions.", 'genres': ['Self-Help', 'Graphic Novel', 'Philosophy'], 'metadata': {'ISBN': '978-1-228-26808-3', 'language': 'Korean', 'edition': 'First'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Into Tides Under the Sky', 'author': 'Rachel Jacobson', 'numberOfPages': 223, 'rating': 1.2, 'publicationYear': 2010, 'summary': "Set in a magical forest, Rachel Jacobson's 'Into Tides Under the Sky' captures the essence of an unraveling conspiracy, with characters struggling through making impossible sacrifices. a philosophical meditation awaits readers.", 'genres': ['Satire', 'Young Adult', 'Philosophy', 'Poetry'], 'metadata': {'ISBN': '978-0-513-57979-8', 'language': 'Italian', 'edition': 'Anniversary Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Beyond Waves in the Universe', 'author': 'Nicholas Snyder', 'numberOfPages': 717, 'rating': 1.9, 'publicationYear': 2002, 'summary': "'Beyond Waves in the Universe' by Nicholas Snyder unfolds in an ancient underwater city, where a journey to the edge of the world sets the stage. Characters navigate defending a fragile hope in an inventive flight of fancy.", 'genres': ['Science Fiction', 'Self-Help', 'Mystery'], 'metadata': {'ISBN': '978-1-4879-8163-1', 'language': 'Chinese', 'edition': 'Limited Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Beneath Operas of Lost Souls', 'author': 'Benjamin Patterson', 'numberOfPages': 724, 'rating': 1.8, 'publicationYear': 1975, 'summary': "Set amidst a hidden library, 'Beneath Operas of Lost Souls' by Benjamin Patterson takes on a battle for the soul, where fulfilling a forgotten prophecy leads to a deeply introspective narrative.", 'genres': ['Poetry', 'Satire'], 'metadata': {'ISBN': '978-0-13-465453-9', 'language': 'Italian', 'edition': 'Anniversary Edition'}, 'isCheckedOut': True, 'borrower': 'Steve Fernandez', 'dueDate': '2024-12-13'}, {'title': 'Through Whispers on the Edge', 'author': 'Michael Olson', 'numberOfPages': 284, 'rating': 1.9, 'publicationYear': 1981, 'summary': "Within the world of a futuristic laboratory, Michael Olson's 'Through Whispers on the Edge' unravels a tale of a fight for honor. Characters must confront rekindling a lost love, resulting in a dark and twisted tale of horror.", 'genres': ['Self-Help', 'Romance'], 'metadata': {'ISBN': '978-0-934153-59-1', 'language': 'Japanese', 'edition': 'Limited Edition'}, 'isCheckedOut': True, 'borrower': 'Kathleen Fisher', 'dueDate': '2024-12-10'}, {'title': 'Across Silence and the Ancient Prophecy', 'author': 'Eric Fitzgerald', 'numberOfPages': 709, 'rating': 1.3, 'publicationYear': 1983, 'summary': "In a barren wasteland, 'Across Silence and the Ancient Prophecy' by Eric Fitzgerald takes readers on a journey through a journey to the afterlife. The characters are challenged by salvaging a shattered trust, creating an inventive flight of fancy.", 'genres': ['Science Fiction', 'Romance', 'Young Adult', 'Poetry'], 'metadata': {'ISBN': '978-1-06-255199-0', 'language': 'Russian', 'edition': "Collector's Edition"}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'In Darkness of Brightness', 'author': 'Nichole Carlson', 'numberOfPages': 433, 'rating': 3.2, 'publicationYear': 1983, 'summary': "Amidst a sprawling metropolis, 'In Darkness of Brightness' by Nichole Carlson delves into a battle against the elements. The characters must overcome rebuilding a kingdom, making this a poignant and bittersweet reflection.", 'genres': ['Graphic Novel', 'Poetry', 'Epic'], 'metadata': {'ISBN': '978-0-592-30845-6', 'language': 'German', 'edition': 'Deluxe Edition'}, 'isCheckedOut': True, 'borrower': 'Robert Trevino', 'dueDate': '2024-11-26'}, {'title': 'A Dreams of the Landscape', 'author': 'Jennifer Richardson', 'numberOfPages': 779, 'rating': 1.5, 'publicationYear': 1992, 'summary': "In a desert wasteland, Jennifer Richardson's 'A Dreams of the Landscape' unravels a gripping tale of a quest for peace. The characters must navigate fulfilling a forgotten promise, creating a dark and brooding story.", 'genres': ['Epic', 'Young Adult', 'Historical Fiction', 'Romance'], 'metadata': {'ISBN': '978-0-303-61952-9', 'language': 'French', 'edition': 'Second'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'In Echoes in the Dark', 'author': 'Rhonda Barton', 'numberOfPages': 578, 'rating': 2.2, 'publicationYear': 2000, 'summary': "In the backdrop of a lost temple, 'In Echoes in the Dark' by Rhonda Barton explores a fight for honor. The characters' journey through unleashing a terrible curse results in a poetic and haunting story.", 'genres': ['Comedy', 'Epic'], 'metadata': {'ISBN': '978-0-236-54474-5', 'language': 'Russian', 'edition': 'Anniversary Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'The Silence of Broken Promises', 'author': 'Jason Kelly', 'numberOfPages': 737, 'rating': 3.7, 'publicationYear': 2013, 'summary': "Jason Kelly's masterpiece, 'The Silence of Broken Promises', is a story set in an isolated lighthouse. Focusing on a struggle against tyranny, the narrative reveals confronting a powerful enemy, delivering a thrilling and action-packed blockbuster.", 'genres': ['Drama', 'Memoir', 'Science Fiction', 'Biography'], 'metadata': {'ISBN': '978-0-7559-3465-2', 'language': 'Arabic', 'edition': 'Special Release'}, 'isCheckedOut': True, 'borrower': 'Matthew Brown', 'dueDate': '2024-12-16'}, {'title': 'From the Wind and the Journey', 'author': 'Barbara Fisher', 'numberOfPages': 611, 'rating': 2.4, 'publicationYear': 2002, 'summary': "In 'From the Wind and the Journey', Barbara Fisher crafts a narrative set in a war-torn kingdom, focusing on a journey to the afterlife. With surviving against all odds, this book is a dark and brooding story.", 'genres': ['History', 'Young Adult'], 'metadata': {'ISBN': '978-1-996786-98-7', 'language': 'German', 'edition': 'Limited Edition'}, 'isCheckedOut': True, 'borrower': 'Rebecca Wagner', 'dueDate': '2024-12-02'}, {'title': 'From Moments of Eternal Night', 'author': 'Monica Dixon', 'numberOfPages': 404, 'rating': 2.3, 'publicationYear': 1985, 'summary': "Set in a hidden library, Monica Dixon's 'From Moments of Eternal Night' captures the essence of a journey through time, with characters struggling through avenging a terrible wrong. a thought-provoking exploration of the human condition awaits readers.", 'genres': ['Thriller', 'Science Fiction'], 'metadata': {'ISBN': '978-1-174-78336-4', 'language': 'Spanish', 'edition': 'Third'}, 'isCheckedOut': True, 'borrower': 'Thomas Kramer', 'dueDate': '2024-12-03'}, {'title': 'A Legends Through Time', 'author': 'Colleen Johns', 'numberOfPages': 621, 'rating': 2.3, 'publicationYear': 1994, 'summary': "'A Legends Through Time' is Colleen Johns's brilliant novel, set in a haunted shack. Exploring a journey of self-discovery, the characters face finding a way home, resulting in a whimsical and enchanting fable.", 'genres': ['Horror', 'Philosophy', 'Mystery'], 'metadata': {'ISBN': '978-0-691-69112-1', 'language': 'French', 'edition': 'Limited Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'In Paths of the Past', 'author': 'Antonio Juarez', 'numberOfPages': 517, 'rating': 1.2, 'publicationYear': 2008, 'summary': "In the backdrop of a moonlit battlefield, 'In Paths of the Past' by Antonio Juarez explores a quest for revenge. The characters' journey through reawakening a lost memory results in a thrilling and action-packed blockbuster.", 'genres': ['Self-Help', 'Horror'], 'metadata': {'ISBN': '978-0-323-81383-9', 'language': 'Italian', 'edition': 'First'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Among Legends and the Last Frontier', 'author': 'Randy Ellis', 'numberOfPages': 756, 'rating': 3.9, 'publicationYear': 1986, 'summary': "'Among Legends and the Last Frontier' is Randy Ellis's brilliant novel, set in a crumbling mansion. Exploring a fight against fate, the characters face unlocking a hidden power, resulting in a deeply introspective narrative.", 'genres': ['Romance', 'Dystopian', 'Self-Help', 'Horror'], 'metadata': {'ISBN': '978-1-55510-079-7', 'language': 'Italian', 'edition': 'Illustrated Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Into Tides of a Lifetime', 'author': 'Jon Hill', 'numberOfPages': 716, 'rating': 1.3, 'publicationYear': 2001, 'summary': "'Into Tides of a Lifetime', a novel by Jon Hill, is set in an ancient underwater city, where a fight against fate unfolds. The characters grapple with defying a cruel fate, creating a gripping tale of suspense.", 'genres': ['Dystopian'], 'metadata': {'ISBN': '978-0-09-408237-3', 'language': 'Spanish', 'edition': 'Deluxe Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Hidden Paths of Eternal Night', 'author': 'Samantha Norris', 'numberOfPages': 579, 'rating': 3.5, 'publicationYear': 2001, 'summary': "In 'Hidden Paths of Eternal Night', Samantha Norris crafts a narrative set in a barren wasteland, focusing on a struggle for power. With defying a tyrant, this book is a suspenseful page-turner.", 'genres': ['Fantasy', 'Science Fiction'], 'metadata': {'ISBN': '978-0-18-203978-4', 'language': 'Arabic', 'edition': 'Special Release'}, 'isCheckedOut': True, 'borrower': 'Bradley Castaneda', 'dueDate': '2024-12-10'}, {'title': 'Within Friendships of the Past', 'author': 'Patrick Harper', 'numberOfPages': 315, 'rating': 2.6, 'publicationYear': 1994, 'summary': "In Patrick Harper's 'Within Friendships of the Past', the setting of a secret base in the Arctic becomes a stage for a journey of self-discovery. The narrative weaves through unleashing a terrible weapon, offering a chilling and atmospheric tale.", 'genres': ['Romance', 'Fantasy', 'Dystopian', 'Drama'], 'metadata': {'ISBN': '978-1-05-715850-0', 'language': 'English', 'edition': 'Anniversary Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Into Moments of Broken Promises', 'author': 'Gerald Hill', 'numberOfPages': 617, 'rating': 4.0, 'publicationYear': 2002, 'summary': "'Into Moments of Broken Promises', written by Gerald Hill, is a story set in a desert wasteland. Delving into a quest for peace, the characters tackle salvaging a shattered trust, creating a tour de force.", 'genres': ['Comedy'], 'metadata': {'ISBN': '978-0-517-99501-3', 'language': 'Chinese', 'edition': 'Second'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Beneath Moments in the Universe', 'author': 'Hector Ramirez', 'numberOfPages': 255, 'rating': 1.1, 'publicationYear': 1977, 'summary': "Hector Ramirez's masterpiece, 'Beneath Moments in the Universe', is a story set in an oasis of art. Focusing on a search for inner peace, the narrative reveals reclaiming a stolen legacy, delivering a rich and immersive world.", 'genres': ['Satire', 'Drama'], 'metadata': {'ISBN': '978-0-624-45317-8', 'language': 'Spanish', 'edition': 'Illustrated Edition'}, 'isCheckedOut': True, 'borrower': 'Jason Blake', 'dueDate': '2024-12-20'}, {'title': 'Hidden Notes of the Future', 'author': 'Christina Hernandez', 'numberOfPages': 441, 'rating': 3.1, 'publicationYear': 1991, 'summary': "Set in an underwater cave, Christina Hernandez's 'Hidden Notes of the Future' captures the essence of a battle for the ages, with characters struggling through unleashing a terrible weapon. a whimsical and enchanting fable awaits readers.", 'genres': ['Comedy', 'Graphic Novel'], 'metadata': {'ISBN': '978-1-284-81716-4', 'language': 'French', 'edition': 'Special Release'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Beneath Light on the Edge', 'author': 'Heather Morales', 'numberOfPages': 698, 'rating': 1.5, 'publicationYear': 2003, 'summary': "Amidst a small, quiet village, 'Beneath Light on the Edge' by Heather Morales delves into a battle for freedom. The characters must overcome salvaging a shattered trust, making this a deeply introspective narrative.", 'genres': ['Dystopian'], 'metadata': {'ISBN': '978-1-5295-8182-9', 'language': 'Japanese', 'edition': 'Limited Edition'}, 'isCheckedOut': True, 'borrower': 'Caroline White', 'dueDate': '2024-12-09'}, {'title': 'Among Mirrors of Eternal Night', 'author': 'Sarah Olson', 'numberOfPages': 694, 'rating': 3.5, 'publicationYear': 2003, 'summary': "In 'Among Mirrors of Eternal Night', Sarah Olson sets a story of a quest for redemption against the vivid backdrop of a lush garden. The characters' struggles with rediscovering a lost friendship result in a deeply moving story.", 'genres': ['Historical Fiction', 'Thriller', 'Graphic Novel'], 'metadata': {'ISBN': '978-0-543-47061-4', 'language': 'Portuguese', 'edition': 'Second'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Within Silence of the Past', 'author': 'Victoria Holt', 'numberOfPages': 591, 'rating': 1.0, 'publicationYear': 1981, 'summary': "In Victoria Holt's 'Within Silence of the Past', set in an isolated lighthouse, readers explore a struggle against destiny. Faced with reconciling a bitter rivalry, the story delivers a masterful blend of tension and hope.", 'genres': ['Non-Fiction', 'Fantasy', 'Historical Fiction', 'Tragedy'], 'metadata': {'ISBN': '978-1-04-230685-5', 'language': 'Arabic', 'edition': 'Second'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Across Silence on the Edge', 'author': 'Lisa Martinez', 'numberOfPages': 372, 'rating': 2.0, 'publicationYear': 1985, 'summary': "Set amidst a small, quiet village, 'Across Silence on the Edge' by Lisa Martinez takes on a fight against fate, where unleashing a terrible curse leads to a tour de force.", 'genres': ['Graphic Novel', 'Epic', 'Self-Help'], 'metadata': {'ISBN': '978-0-06-126190-9', 'language': 'Korean', 'edition': 'Second'}, 'isCheckedOut': True, 'borrower': 'Debbie Thornton', 'dueDate': '2024-11-25'}, {'title': 'From Ghosts and the Journey', 'author': 'Toni Figueroa', 'numberOfPages': 303, 'rating': 4.0, 'publicationYear': 1975, 'summary': "'From Ghosts and the Journey' by Toni Figueroa immerses readers in a desert wasteland, where a clash of cultures collides with escaping from relentless pursuers, delivering a thrilling and fast-paced adventure.", 'genres': ['Romance', 'Philosophy', 'Self-Help', 'Satire'], 'metadata': {'ISBN': '978-0-939649-88-4', 'language': 'French', 'edition': 'Second'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Among Moons of the Landscape', 'author': 'Phillip Garner', 'numberOfPages': 760, 'rating': 4.6, 'publicationYear': 2007, 'summary': "In a lush garden, Phillip Garner's 'Among Moons of the Landscape' tells a tale of a battle for freedom. The narrative unfolds as the characters navigate unlocking a hidden power, offering a lighthearted romp.", 'genres': ['Graphic Novel', 'Horror', 'Poetry', 'Dystopian'], 'metadata': {'ISBN': '978-0-01-177499-2', 'language': 'Japanese', 'edition': 'Illustrated Edition'}, 'isCheckedOut': True, 'borrower': 'Richard Ford', 'dueDate': '2024-11-29'}, {'title': 'From Acquaintances and the Ancient Prophecy', 'author': 'John Williams', 'numberOfPages': 622, 'rating': 4.2, 'publicationYear': 2008, 'summary': "'From Acquaintances and the Ancient Prophecy' by John Williams immerses readers in a lost temple, where a battle for the future collides with reviving a shattered empire, delivering an unforgettable and immersive experience.", 'genres': ['Non-Fiction'], 'metadata': {'ISBN': '978-1-888696-43-1', 'language': 'Arabic', 'edition': 'Deluxe Edition'}, 'isCheckedOut': True, 'borrower': 'Steven Blankenship', 'dueDate': '2024-12-12'}, {'title': 'Beneath Mirrors of Tomorrow', 'author': 'Dana Lopez', 'numberOfPages': 594, 'rating': 4.4, 'publicationYear': 2013, 'summary': "Dana Lopez's masterpiece, 'Beneath Mirrors of Tomorrow', is a story set in a desert wasteland. Focusing on a search for answers, the narrative reveals questioning their identity, delivering a poignant and bittersweet reflection.", 'genres': ['Comedy', 'Horror', 'Poetry'], 'metadata': {'ISBN': '978-1-5493-1582-4', 'language': 'Arabic', 'edition': 'Second'}, 'isCheckedOut': True, 'borrower': 'Anna Sharp', 'dueDate': '2024-11-26'}, {'title': 'The Stars of Tomorrow', 'author': 'Julie Wright', 'numberOfPages': 584, 'rating': 1.7, 'publicationYear': 1982, 'summary': "In Julie Wright's 'The Stars of Tomorrow', the setting of a haunted shack becomes a stage for a search for purpose. The narrative weaves through defending a dream, offering a heartrending saga of love and loss.", 'genres': ['Mystery'], 'metadata': {'ISBN': '978-0-903964-68-5', 'language': 'Russian', 'edition': 'Illustrated Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'The Echoes of the Heart', 'author': 'Miguel Wagner', 'numberOfPages': 314, 'rating': 2.5, 'publicationYear': 1979, 'summary': "Within a sprawling museum, 'The Echoes of the Heart' by Miguel Wagner captures the spirit of a forbidden love affair. The story builds as characters confront rekindling a lost passion, offering a whimsical and enchanting fable.", 'genres': ['Adventure', 'Fantasy', 'Comedy', 'Science Fiction'], 'metadata': {'ISBN': '978-1-69653-095-8', 'language': 'Portuguese', 'edition': 'Limited Edition'}, 'isCheckedOut': True, 'borrower': 'Priscilla Joseph', 'dueDate': '2024-12-01'}, {'title': 'Across the Wind in the Dark', 'author': 'Zachary Marshall', 'numberOfPages': 777, 'rating': 4.6, 'publicationYear': 1990, 'summary': "Amidst a lost temple, 'Across the Wind in the Dark' by Zachary Marshall delves into a journey of self-discovery. The characters must overcome rebuilding lost trust, making this a dazzling epic of wonder and imagination.", 'genres': ['Historical Fiction'], 'metadata': {'ISBN': '978-0-06-738327-8', 'language': 'Chinese', 'edition': 'Second'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Hidden Darkness that Changed Everything', 'author': 'Michael Rocha', 'numberOfPages': 349, 'rating': 5.0, 'publicationYear': 2014, 'summary': "Set in a forgotten metropolis, 'Hidden Darkness that Changed Everything' by Michael Rocha dives into a struggle against darkness. Facing facing their greatest fears, this is a light and breezy beach read.", 'genres': ['Mystery', 'Fantasy', 'Biography'], 'metadata': {'ISBN': '978-0-308-04093-9', 'language': 'German', 'edition': 'Deluxe Edition'}, 'isCheckedOut': True, 'borrower': 'John Klein MD', 'dueDate': '2024-11-25'}, {'title': 'In Legends of the Future', 'author': 'Amber May', 'numberOfPages': 483, 'rating': 1.7, 'publicationYear': 2005, 'summary': "In 'In Legends of the Future', Amber May sets a story of an exploration of forbidden knowledge against the vivid backdrop of an oasis of art. The characters' struggles with questioning their identity result in a thrilling and fast-paced adventure.", 'genres': ['Non-Fiction', 'Fantasy', 'Satire', 'Mystery'], 'metadata': {'ISBN': '978-0-06-364971-2', 'language': 'Russian', 'edition': 'Anniversary Edition'}, 'isCheckedOut': True, 'borrower': 'Alicia Torres', 'dueDate': '2024-12-17'}, {'title': 'Hidden Echoes of a Lifetime', 'author': 'Joseph Dominguez', 'numberOfPages': 275, 'rating': 2.8, 'publicationYear': 2024, 'summary': "In 'Hidden Echoes of a Lifetime', Joseph Dominguez sets a story of a journey of self-discovery against the vivid backdrop of a distant alien world. The characters' struggles with finding a way home result in a rich and evocative tapestry of words.", 'genres': ['Philosophy', 'Young Adult', 'Mystery', 'Biography'], 'metadata': {'ISBN': '978-1-895170-83-2', 'language': 'German', 'edition': 'Deluxe Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Through Tides of Yesterday', 'author': 'Travis Murphy', 'numberOfPages': 535, 'rating': 2.6, 'publicationYear': 1996, 'summary': "In 'Through Tides of Yesterday', Travis Murphy weaves a tale set in a haunted shack, centered around a clash of cultures. With fulfilling a forgotten destiny, the story becomes a deeply moving story.", 'genres': ['Satire'], 'metadata': {'ISBN': '978-0-237-35730-6', 'language': 'Korean', 'edition': 'Illustrated Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'In Legends of Lost Souls', 'author': 'Frank Washington', 'numberOfPages': 439, 'rating': 1.4, 'publicationYear': 1988, 'summary': "In Frank Washington's 'In Legends of Lost Souls', the setting of a hidden library becomes a stage for a fight for justice. The narrative weaves through unleashing a terrible weapon, offering a dazzling epic of wonder and imagination.", 'genres': ['Tragedy'], 'metadata': {'ISBN': '978-0-220-33316-4', 'language': 'English', 'edition': 'Illustrated Edition'}, 'isCheckedOut': True, 'borrower': 'Dominic Carter', 'dueDate': '2024-12-20'}, {'title': 'From the Horizon of Lost Souls', 'author': 'Jeffery Parker', 'numberOfPages': 627, 'rating': 3.4, 'publicationYear': 1984, 'summary': "Set amidst a remote island, 'From the Horizon of Lost Souls' by Jeffery Parker takes on a journey through time, where fulfilling a forgotten destiny leads to a heartrending saga of love and loss.", 'genres': ['Horror'], 'metadata': {'ISBN': '978-1-4489-3322-8', 'language': 'Japanese', 'edition': 'Limited Edition'}, 'isCheckedOut': True, 'borrower': 'Robin Pham', 'dueDate': '2024-12-12'}, {'title': 'Through Operas of Lost Souls', 'author': 'Tyler Fitzgerald', 'numberOfPages': 427, 'rating': 4.0, 'publicationYear': 2012, 'summary': "Within a secret base in the Arctic, 'Through Operas of Lost Souls' by Tyler Fitzgerald captures the spirit of a quest for the truth. The story builds as characters confront salvaging a shattered trust, offering a chilling and atmospheric tale.", 'genres': ['Memoir', 'Drama', 'Philosophy'], 'metadata': {'ISBN': '978-0-283-73842-5', 'language': 'Japanese', 'edition': 'Anniversary Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Among Legends of the Future', 'author': 'Mark Vasquez', 'numberOfPages': 366, 'rating': 2.3, 'publicationYear': 2013, 'summary': "'Among Legends of the Future' by Mark Vasquez immerses readers in an underwater cave, where a battle for the future collides with escaping from relentless pursuers, delivering a deep character study.", 'genres': ['Poetry'], 'metadata': {'ISBN': '978-0-7792-2720-4', 'language': 'Arabic', 'edition': 'First'}, 'isCheckedOut': True, 'borrower': 'Susan Williams', 'dueDate': '2024-12-02'}, {'title': 'Across Stars and the Unknown', 'author': 'Scott Russell', 'numberOfPages': 602, 'rating': 4.6, 'publicationYear': 2011, 'summary': "'Across Stars and the Unknown' by Scott Russell is set in a remote island and tells a story of a quest for vengeance. The protagonists face defeating a powerful adversary, in a tale that delivers a tour de force.", 'genres': ['Dystopian'], 'metadata': {'ISBN': '978-1-916911-10-9', 'language': 'German', 'edition': 'Illustrated Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'From Ashes of the Heart', 'author': 'Jason Duncan', 'numberOfPages': 198, 'rating': 2.9, 'publicationYear': 1994, 'summary': "Set against a remote island, 'From Ashes of the Heart' by Jason Duncan unveils a quest for redemption. The story builds through defying a tyrant, offering a deep character study.", 'genres': ['Horror'], 'metadata': {'ISBN': '978-0-8401-0949-1', 'language': 'Italian', 'edition': 'Anniversary Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Beneath Waves Through Time', 'author': 'Kristin Garrett', 'numberOfPages': 296, 'rating': 3.2, 'publicationYear': 1994, 'summary': "In 'Beneath Waves Through Time', Kristin Garrett crafts a narrative set in a bustling marketplace, focusing on a journey of self-discovery. With solving a baffling mystery, this book is a pulse-pounding thrill ride.", 'genres': ['Biography', 'History', 'Poetry', 'Thriller'], 'metadata': {'ISBN': '978-0-7701-5322-9', 'language': 'German', 'edition': 'Anniversary Edition'}, 'isCheckedOut': True, 'borrower': 'Steven Smith', 'dueDate': '2024-11-29'}, {'title': 'Beyond Dreams and Forgotten Worlds', 'author': 'James Lee', 'numberOfPages': 395, 'rating': 1.7, 'publicationYear': 2007, 'summary': "'Beyond Dreams and Forgotten Worlds' by James Lee immerses readers in a remote island, where a search for answers collides with unlocking a hidden power, delivering a evocative tapestry of emotions.", 'genres': ['Biography', 'Non-Fiction', 'Satire'], 'metadata': {'ISBN': '978-1-06-454125-8', 'language': 'Russian', 'edition': 'Limited Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Under Whispers of Broken Promises', 'author': 'Eduardo Gibbs', 'numberOfPages': 375, 'rating': 4.6, 'publicationYear': 2002, 'summary': "In Eduardo Gibbs's 'Under Whispers of Broken Promises', the setting of a hidden library becomes a stage for a clash of cultures. The narrative weaves through facing their greatest fears, offering a thought-provoking exploration of the human condition.", 'genres': ['Philosophy', 'Self-Help'], 'metadata': {'ISBN': '978-0-325-25094-6', 'language': 'French', 'edition': 'Limited Edition'}, 'isCheckedOut': True, 'borrower': 'Charles Barber', 'dueDate': '2024-12-14'}, {'title': 'Among Dreams and the Journey', 'author': 'Melissa Wyatt', 'numberOfPages': 574, 'rating': 4.7, 'publicationYear': 2009, 'summary': "'Among Dreams and the Journey' by Melissa Wyatt unfolds in a crumbling mansion, where a struggle against time sets the stage. Characters navigate facing their greatest fears in a dark and brooding story.", 'genres': ['Science Fiction', 'Romance', 'Dystopian'], 'metadata': {'ISBN': '978-1-177-04903-0', 'language': 'Russian', 'edition': 'Special Release'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Beyond Notes of Tomorrow', 'author': 'Elizabeth Perez', 'numberOfPages': 436, 'rating': 2.6, 'publicationYear': 1984, 'summary': "Within the world of a forgotten metropolis, Elizabeth Perez's 'Beyond Notes of Tomorrow' unravels a tale of a race to uncover ancient secrets. Characters must confront defying a cruel god, resulting in a light and breezy beach read.", 'genres': ['Fantasy', 'Memoir', 'Non-Fiction'], 'metadata': {'ISBN': '978-1-992435-32-2', 'language': 'Russian', 'edition': 'Deluxe Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Within Legends and the Journey', 'author': 'Peter Dixon', 'numberOfPages': 741, 'rating': 3.1, 'publicationYear': 2019, 'summary': "Within a bustling marketplace, 'Within Legends and the Journey' by Peter Dixon captures the spirit of a fight for survival. The story builds as characters confront overcoming their own doubts, offering a gripping tale of suspense.", 'genres': ['Romance', 'Adventure', 'Philosophy'], 'metadata': {'ISBN': '978-0-03-199377-9', 'language': 'Russian', 'edition': 'Limited Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Under Secrets Under the Sky', 'author': 'Ryan Phillips', 'numberOfPages': 360, 'rating': 2.9, 'publicationYear': 2011, 'summary': "In Ryan Phillips's 'Under Secrets Under the Sky', set in a barren wasteland, readers explore a race to uncover ancient secrets. Faced with unleashing a terrible weapon, the story delivers a thrilling and fast-paced adventure.", 'genres': ['History', 'Non-Fiction', 'Philosophy', 'Poetry'], 'metadata': {'ISBN': '978-1-101-97526-8', 'language': 'English', 'edition': 'Third'}, 'isCheckedOut': True, 'borrower': 'Lisa Turner', 'dueDate': '2024-12-09'}, {'title': 'Hidden Secrets and the Ancient Prophecy', 'author': 'Julie Thompson', 'numberOfPages': 700, 'rating': 1.3, 'publicationYear': 2016, 'summary': "In Julie Thompson's 'Hidden Secrets and the Ancient Prophecy', the setting of an isolated lighthouse becomes a stage for an unraveling conspiracy. The narrative weaves through navigating treacherous terrain, offering a thrilling and action-packed blockbuster.", 'genres': ['Dystopian'], 'metadata': {'ISBN': '978-0-05-478884-0', 'language': 'Russian', 'edition': 'Limited Edition'}, 'isCheckedOut': True, 'borrower': 'Haley Macias', 'dueDate': '2024-11-22'}, {'title': 'Into Moments of the Landscape', 'author': 'Wesley Hamilton', 'numberOfPages': 430, 'rating': 2.9, 'publicationYear': 1999, 'summary': "Wesley Hamilton's masterpiece, 'Into Moments of the Landscape', is a story set in a desert wasteland. Focusing on a quest for the truth, the narrative reveals navigating treacherous terrain, delivering a light and breezy beach read.", 'genres': ['Satire', 'Science Fiction', 'History'], 'metadata': {'ISBN': '978-1-4294-2654-1', 'language': 'Korean', 'edition': "Collector's Edition"}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Hidden the Horizon of the Heart', 'author': 'Ashley Beltran', 'numberOfPages': 316, 'rating': 2.0, 'publicationYear': 1981, 'summary': "Within the world of an isolated lighthouse, Ashley Beltran's 'Hidden the Horizon of the Heart' unravels a tale of a fight for honor. Characters must confront avenging a terrible wrong, resulting in a deep character study.", 'genres': ['Drama', 'Romance', 'Philosophy'], 'metadata': {'ISBN': '978-1-143-40842-7', 'language': 'Arabic', 'edition': 'First'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Into Darkness on the Edge', 'author': 'Caitlin Leon', 'numberOfPages': 449, 'rating': 2.2, 'publicationYear': 2004, 'summary': "In Caitlin Leon's 'Into Darkness on the Edge', set in a magical forest, readers explore a search for inner peace. Faced with defying a cruel fate, the story delivers an unforgettable and immersive experience.", 'genres': ['Epic', 'Fantasy'], 'metadata': {'ISBN': '978-0-89557-521-0', 'language': 'German', 'edition': 'Second'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Into Darkness on the Edge', 'author': 'Theodore Brown', 'numberOfPages': 723, 'rating': 2.0, 'publicationYear': 2017, 'summary': "Set in a magical forest, 'Into Darkness on the Edge' by Theodore Brown dives into an unraveling conspiracy. Facing solving a baffling mystery, this is a gripping tale of suspense.", 'genres': ['Thriller', 'Mystery', 'Tragedy', 'Romance'], 'metadata': {'ISBN': '978-0-353-53711-8', 'language': 'Spanish', 'edition': 'Anniversary Edition'}, 'isCheckedOut': True, 'borrower': 'Elizabeth Casey', 'dueDate': '2024-11-25'}, {'title': 'A Moons that Changed Everything', 'author': 'Brian Torres', 'numberOfPages': 795, 'rating': 1.4, 'publicationYear': 1983, 'summary': "Amidst a distant alien world, 'A Moons that Changed Everything' by Brian Torres delves into a journey to the afterlife. The characters must overcome reclaiming a stolen legacy, making this a heartwarming journey.", 'genres': ['Biography', 'Adventure', 'Philosophy', 'Epic'], 'metadata': {'ISBN': '978-0-7893-9962-5', 'language': 'Italian', 'edition': 'Second'}, 'isCheckedOut': True, 'borrower': 'Robert Ross', 'dueDate': '2024-11-22'}, {'title': 'Into Silence in the Universe', 'author': 'Hayley Thompson', 'numberOfPages': 524, 'rating': 2.1, 'publicationYear': 2008, 'summary': "'Into Silence in the Universe' is Hayley Thompson's latest masterpiece set in a distant alien world. With a focus on a quest for redemption, the story brings to life deciphering cryptic messages, delivering a lighthearted romp.", 'genres': ['Historical Fiction', 'Self-Help', 'Dystopian'], 'metadata': {'ISBN': '978-1-57695-980-0', 'language': 'English', 'edition': 'Illustrated Edition'}, 'isCheckedOut': True, 'borrower': 'Jeanne Tucker', 'dueDate': '2024-12-08'}, {'title': 'Beneath Phantoms of the Past', 'author': 'Kevin Santiago', 'numberOfPages': 401, 'rating': 1.4, 'publicationYear': 2018, 'summary': "'Beneath Phantoms of the Past' by Kevin Santiago takes place in a hidden library, where an unsolved mystery shapes the characters' journey through reclaiming a stolen legacy. a deep character study.", 'genres': ['Biography', 'Adventure', 'Satire'], 'metadata': {'ISBN': '978-1-4850-4091-0', 'language': 'French', 'edition': 'Second'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Through Ashes of the Future', 'author': 'Rebecca Watson', 'numberOfPages': 385, 'rating': 1.4, 'publicationYear': 1981, 'summary': "Set against a vibrant carnival, 'Through Ashes of the Future' by Rebecca Watson unveils a fight for honor. The story builds through making impossible sacrifices, offering a poignant and bittersweet reflection.", 'genres': ['History', 'Romance', 'Satire', 'Poetry'], 'metadata': {'ISBN': '978-1-5380-0152-3', 'language': 'Portuguese', 'edition': 'First'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Against Whispers of Yesterday', 'author': 'Megan Reed', 'numberOfPages': 325, 'rating': 3.0, 'publicationYear': 1994, 'summary': "'Against Whispers of Yesterday', written by Megan Reed, paints a vivid picture of a barren wasteland. Exploring a struggle against tyranny, the story revolves around confronting a powerful enemy, offering a dark and twisted tale of horror.", 'genres': ['Epic'], 'metadata': {'ISBN': '978-0-02-315141-5', 'language': 'Portuguese', 'edition': 'Third'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Under Waves of the Landscape', 'author': 'Aaron Morris', 'numberOfPages': 774, 'rating': 2.4, 'publicationYear': 2019, 'summary': "'Under Waves of the Landscape' is Aaron Morris's latest masterpiece set in a lost temple. With a focus on a battle for the ages, the story brings to life defending a dream, delivering a pulse-pounding thrill ride.", 'genres': ['Mystery', 'Science Fiction', 'Self-Help'], 'metadata': {'ISBN': '978-0-493-30631-5', 'language': 'Italian', 'edition': 'Second'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'In Waves of a Lifetime', 'author': 'Richard Rocha', 'numberOfPages': 591, 'rating': 2.2, 'publicationYear': 2008, 'summary': "In 'In Waves of a Lifetime', Richard Rocha sets a story of a fight for love against the vivid backdrop of a distant alien world. The characters' struggles with unleashing a terrible force result in a philosophical meditation.", 'genres': ['Romance', 'Self-Help'], 'metadata': {'ISBN': '978-1-393-88060-8', 'language': 'English', 'edition': 'Deluxe Edition'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'In Stars of Eternal Night', 'author': 'Chris Byrd', 'numberOfPages': 719, 'rating': 4.0, 'publicationYear': 1983, 'summary': "'In Stars of Eternal Night', written by Chris Byrd, paints a vivid picture of a secret base in the Arctic. Exploring a fight for honor, the story revolves around challenging a corrupt system, offering a heartrending saga of love and loss.", 'genres': ['Fantasy', 'History', 'Philosophy'], 'metadata': {'ISBN': '978-1-09-264432-7', 'language': 'German', 'edition': "Collector's Edition"}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Into Reflections and the Journey', 'author': 'Daniel Wheeler', 'numberOfPages': 428, 'rating': 3.1, 'publicationYear': 2005, 'summary': "In the backdrop of a mysterious castle, 'Into Reflections and the Journey' by Daniel Wheeler explores a battle for the ages. The characters' journey through rebuilding a kingdom results in an uplifting tale.", 'genres': ['Thriller', 'Mystery', 'History'], 'metadata': {'ISBN': '978-1-4791-5224-7', 'language': 'Korean', 'edition': 'Third'}, 'isCheckedOut': False, 'borrower': None, 'dueDate': None}, {'title': 'Within Voices of the Past', 'author': 'Cynthia Rivera', 'numberOfPages': 435, 'rating': 1.2, 'publicationYear': 2004, 'summary': "In 'Within Voices of the Past', Cynthia Rivera crafts a narrative set in a barren wasteland, focusing on an unraveling conspiracy. With redeeming a shameful past, this book is a whimsical and enchanting fable.", 'genres': ['Poetry'], 'metadata': {'ISBN': '978-1-59334-122-0', 'language': 'German', 'edition': 'Anniversary Edition'}, 'isCheckedOut': True, 'borrower': 'Allison Hernandez', 'dueDate': '2024-11-28'}]

## Add a function to connect to your database


In [ ]:
def connect_to_database() -> Database:
    """
    Connects to a DataStax Astra database.
    This function retrieves the database endpoint and application token from the
    environment variables `ASTRA_DB_API_ENDPOINT` and `ASTRA_DB_APPLICATION_TOKEN`.

    Returns:
        Database: An instance of the connected database.

    Raises:
        RuntimeError: If the environment variables `ASTRA_DB_API_ENDPOINT` or
        `ASTRA_DB_APPLICATION_TOKEN` are not defined.
    """
    endpoint = userdata.get("ASTRA_DB_API_ENDPOINT")
    token = userdata.get("ASTRA_DB_APPLICATION_TOKEN")

    if not token or not endpoint:
        raise RuntimeError(
            "Environment variables ASTRA_DB_API_ENDPOINT and ASTRA_DB_APPLICATION_TOKEN must be defined"
        )

    # Create an instance of the `DataAPIClient` class.
    client = DataAPIClient()

    # Get the database specified by your endpoint while providing the token.
    database = client.get_database(
        endpoint,
        token=token,
        # keyspace="my_keyspace",  # omitted if using defaults
    )

    print(f"Connected to database {database.info().name}")
    return database

## Add a function to create a collection

In [ ]:
def create_collection(database: Database, collection_name: str) -> Collection:
    """
    Creates a collection in the specified database with vectorization enabled.
    The collection will use Nvidia's NV-Embed-QA embedding model
    to generate vector embeddings for data in the collection.

    Args:
        database (Database): The instantiated object that represents the
            database where the collection will be created.
        collection_name (str): The name of the collection to create.

    Returns:
        Collection: The created collection.
    """
    collection = database.create_collection(
        collection_name,
        definition=CollectionDefinition(
            vector=CollectionVectorOptions(
                metric=VectorMetric.COSINE,
                service=VectorServiceOptions(
                    provider="nvidia",
                    model_name="NV-Embed-QA",
                )
            )
        )
    )

    print(f"Created collection {collection.full_name}")
    return collection

## Add a function to insert data

In [ ]:
def upload_json_data(
    collection: Collection,
    data_list: List[Dict[str, Any]],
    embedding_string_creator: Callable[[Dict[str, Any]], str],
) -> None:
    """
     Uploads data from a file containing a JSON array to the specified collection.
     For each piece of data, a $vectorize field is added. The $vectorize value is
     a string from which vector embeddings will be generated.

    Args:
        collection (Collection): The instantiated object that represents the
            collection to upload data to.
        data_list: a list of documents to insert, each being a Python dict.
        embedding_string_creator (Callable[[Dict[str, Any]], str]): A function to create
            the string for which vector embeddings will be generated.
    """
    # Add a $vectorize field to each piece of data.
    documents = [
        {
            **data,
            "$vectorize": embedding_string_creator(data),
        }
        for data in data_list
    ]

    # Upload the data.
    inserted = collection.insert_many(documents)
    print(f"Inserted {len(inserted.inserted_ids)} items.")

## Insert data to your database

In [ ]:
database = connect_to_database()

collection = create_collection(database, "quickstart_collection")

upload_json_data(
    collection,
    json_data,
    lambda data: (
        f"summary: {data['summary']} | "
        f"genres: {', '.join(data['genres'])}"
    ),
)

## Find data in your database

In [ ]:
database = connect_to_database()

collection = database.get_collection("quickstart_collection")

# Find documents that match a filter
print("\nFinding books with rating greater than 4.7...")

duration_cursor = collection.find({"rating": {"$gt": 4.7}})

for document in duration_cursor:
    print(f"{document['title']} is rated {document['rating']}")

# Perform a vector search to find the closest match to a search string
print("\nUsing vector search to find a single scary novel...")

single_vector_match = collection.find_one(sort={"$vectorize": "A scary novel"})

print(f"{single_vector_match['title']} is a scary novel")

# Combine a filter, vector search, and projection to find the 3 books with
# more than 400 pages that are the closest matches to a search string,
# and just return the title and author
print("\nUsing filters and vector search to find 3 books with more than 400 pages that are set in the arctic, returning just the title and author...")

vector_cursor = collection.find(
    {"numberOfPages": {"$gt": 400}},
    sort={"$vectorize": "A book set in the arctic"},
    limit=3,
    projection={"title": True, "author": True},
)

for document in vector_cursor:
    print(document)